# BaseLine Modeling

## 1. setting modules

In [1]:
# load modules
import numpy as np
import pandas as pd

# split
from sklearn.model_selection import train_test_split

# models 
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# metrics
from sklearn.metrics import mean_squared_log_error

# optuna
import optuna

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# load data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
# 필요한 데이터만 가져오기
use_train = train[['사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '시군구', '사망자수']]
use_test = test[test.columns.difference(['ID'])]

In [4]:
use_train

,사고일시,요일,기상상태,도로형태,노면상태,사고유형,시군구,사망자수
0,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,대구광역시 중구 대신동,0
1,2019-01-01 00,화요일,흐림,단일로 - 기타,건조,차대사람,대구광역시 달서구 감삼동,0
2,2019-01-01 01,화요일,맑음,단일로 - 기타,건조,차대사람,대구광역시 수성구 두산동,0
3,2019-01-01 02,화요일,맑음,단일로 - 기타,건조,차대차,대구광역시 북구 복현동,0
4,2019-01-01 04,화요일,맑음,단일로 - 기타,건조,차대차,대구광역시 동구 신암동,0
...,...,...,...,...,...,...,...,...
39604,2021-12-31 19,금요일,맑음,교차로 - 교차로안,건조,차대차,대구광역시 수성구 수성동3가,0
39605,2021-12-31 19,금요일,맑음,단일로 - 기타,건조,차대차,대구광역시 달서구 상인동,0
39606,2021-12-31 21,금요일,맑음,교차로 - 교차로안,건조,차대차,대구광역시 달서구 월성동,0
39607,2021-12-31 22,금요일,맑음,기타 - 기타,건조,차대차,대구광역시 달서구 장동,0


In [5]:
use_test

,기상상태,노면상태,도로형태,사고유형,사고일시,시군구,요일
0,맑음,건조,교차로 - 교차로안,차대사람,2022-01-01 01,대구광역시 수성구 상동,토요일
1,맑음,건조,단일로 - 기타,차대사람,2022-01-01 01,대구광역시 수성구 지산동,토요일
2,맑음,건조,교차로 - 교차로안,차대차,2022-01-01 04,대구광역시 수성구 수성동2가,토요일
3,맑음,건조,단일로 - 기타,차대차,2022-01-01 04,대구광역시 수성구 신매동,토요일
4,맑음,건조,교차로 - 교차로안,차대차,2022-01-01 06,대구광역시 달서구 감삼동,토요일
...,...,...,...,...,...,...,...
10958,맑음,건조,단일로 - 터널,차대차,2022-12-31 18,대구광역시 남구 대명동,토요일
10959,맑음,건조,단일로 - 기타,차대차,2022-12-31 18,대구광역시 수성구 시지동,토요일
10960,맑음,건조,단일로 - 기타,차대차,2022-12-31 20,대구광역시 수성구 연호동,토요일
10961,맑음,건조,교차로 - 교차로부근,차대차,2022-12-31 20,대구광역시 수성구 범물동,토요일


## 2. data preprocessing

In [6]:
# one hot encoding
onehot = pd.get_dummies(use_train[['기상상태', '도로형태', '노면상태', '사고유형']], drop_first=True)
use_train2 = pd.concat([use_train, onehot], axis = 1).drop(['기상상태', '도로형태', '노면상태', '사고유형'], axis = 1)

onehot_test = pd.get_dummies(test[['기상상태', '도로형태', '노면상태', '사고유형']], drop_first=True)
use_test2 = pd.concat([use_test, onehot_test], axis = 1).drop(['기상상태', '도로형태', '노면상태', '사고유형'], axis = 1)

In [7]:
use_train2

,사고일시,요일,시군구,사망자수,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_안개,기상상태_흐림,도로형태_교차로 - 교차로안,...,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대차,사고유형_차량단독
0,2019-01-01 00,화요일,대구광역시 중구 대신동,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2019-01-01 00,화요일,대구광역시 달서구 감삼동,0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,2019-01-01 01,화요일,대구광역시 수성구 두산동,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2019-01-01 02,화요일,대구광역시 북구 복현동,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,2019-01-01 04,화요일,대구광역시 동구 신암동,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,2021-12-31 19,금요일,대구광역시 수성구 수성동3가,0,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
39605,2021-12-31 19,금요일,대구광역시 달서구 상인동,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
39606,2021-12-31 21,금요일,대구광역시 달서구 월성동,0,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
39607,2021-12-31 22,금요일,대구광역시 달서구 장동,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [8]:
use_test2

,사고일시,시군구,요일,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_흐림,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,...,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대차,사고유형_차량단독
0,2022-01-01 01,대구광역시 수성구 상동,토요일,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2022-01-01 01,대구광역시 수성구 지산동,토요일,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2022-01-01 04,대구광역시 수성구 수성동2가,토요일,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,2022-01-01 04,대구광역시 수성구 신매동,토요일,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,2022-01-01 06,대구광역시 달서구 감삼동,토요일,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,2022-12-31 18,대구광역시 남구 대명동,토요일,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
10959,2022-12-31 18,대구광역시 수성구 시지동,토요일,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
10960,2022-12-31 20,대구광역시 수성구 연호동,토요일,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
10961,2022-12-31 20,대구광역시 수성구 범물동,토요일,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [9]:
# 요일 월~금: 0 토~일:1
use_train2['주말'] = np.where(use_train2['요일'].isin(['월요일', '화요일', '수요일', '목요일', '금요일']), 0, 1)
use_train3 = use_train2.drop(['요일'], axis = 1)

use_test2['주말'] = np.where(use_test2['요일'].isin(['월요일', '화요일', '수요일', '목요일', '금요일']), 0, 1)
use_test3 = use_test2.drop(['요일'], axis = 1)

In [10]:
use_train3

,사고일시,시군구,사망자수,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_안개,기상상태_흐림,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,...,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대차,사고유형_차량단독,주말
0,2019-01-01 00,대구광역시 중구 대신동,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
1,2019-01-01 00,대구광역시 달서구 감삼동,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,0
2,2019-01-01 01,대구광역시 수성구 두산동,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,2019-01-01 02,대구광역시 북구 복현동,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0
4,2019-01-01 04,대구광역시 동구 신암동,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,2021-12-31 19,대구광역시 수성구 수성동3가,0,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,0
39605,2021-12-31 19,대구광역시 달서구 상인동,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0
39606,2021-12-31 21,대구광역시 달서구 월성동,0,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,0
39607,2021-12-31 22,대구광역시 달서구 장동,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0


In [11]:
use_test3

,사고일시,시군구,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_흐림,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,...,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대차,사고유형_차량단독,주말
0,2022-01-01 01,대구광역시 수성구 상동,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,2022-01-01 01,대구광역시 수성구 지산동,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,2022-01-01 04,대구광역시 수성구 수성동2가,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,1
3,2022-01-01 04,대구광역시 수성구 신매동,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1
4,2022-01-01 06,대구광역시 달서구 감삼동,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,2022-12-31 18,대구광역시 남구 대명동,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1
10959,2022-12-31 18,대구광역시 수성구 시지동,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1
10960,2022-12-31 20,대구광역시 수성구 연호동,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1
10961,2022-12-31 20,대구광역시 수성구 범물동,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1


In [12]:
# 사고일시 나누기

## date type data
use_train3['사고일시'] = pd.to_datetime(use_train3['사고일시'])
use_test3['사고일시'] = pd.to_datetime(use_test3['사고일시'])

In [13]:
# split year, month, day, hour
use_train3['year'] = use_train3['사고일시'].dt.year
use_train3['month'] = use_train3['사고일시'].dt.month
use_train3['day'] = use_train3['사고일시'].dt.day
use_train3['hour'] = use_train3['사고일시'].dt.hour

use_test3['year'] = use_test3['사고일시'].dt.year
use_test3['month'] = use_test3['사고일시'].dt.month
use_test3['day'] = use_test3['사고일시'].dt.day
use_test3['hour'] = use_test3['사고일시'].dt.hour

In [14]:
# 시군구 나누기
use_train3[['시', '구', '동가']] = use_train3['시군구'].str.split(' ', expand = True)
use_train4 = use_train3.drop(['사고일시', '시군구', '시', 'year'], axis = 1)

use_test3[['시', '구', '동가']] = use_test3['시군구'].str.split(' ', expand = True)
use_test4 = use_test3.drop(['사고일시', '시군구', '시', 'year'], axis = 1)

In [15]:
# type change
use_train4['구'] = use_train4['구'].astype('category')
use_train4['동가'] = use_train4['동가'].astype('category')

use_test4['구'] = use_test4['구'].astype('category')
use_test4['동가'] = use_test4['동가'].astype('category')

In [16]:
# 구와 동가 제거
use_train5 = use_train4[use_train4.columns.difference(['구', '동가'])]
use_test5 = use_test4[use_test4.columns.difference(['구', '동가'])]

In [17]:
def wrangiling(data_set):
    print("==================")
    print("       Shape      ")
    print("==================")

    display(data_set.shape)

    print("==================")
    print("        Info      ")
    print("==================")

    display(data_set.info())

    print("==================")
    print("      Columns     ")
    print("==================")
    
    display(data_set.columns)

    print("==================")
    print("        NaN       ")
    print("==================")

    display(data_set.isna().sum())

    print("==================")
    print("     Duplicated   ")
    print("==================")
    
    display(data_set[data_set.duplicated()])

    print("==================")
    print("    Description   ")
    print("==================")

    display(data_set.describe())

    print("==================")
    print("      Unique      ")
    print("==================")

    display(data_set.nunique())

In [18]:
wrangiling(use_train5)

       Shape      


(39609, 27)

        Info      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   day                   39609 non-null  int32
 1   hour                  39609 non-null  int32
 2   month                 39609 non-null  int32
 3   기상상태_눈                39609 non-null  bool 
 4   기상상태_맑음               39609 non-null  bool 
 5   기상상태_비                39609 non-null  bool 
 6   기상상태_안개               39609 non-null  bool 
 7   기상상태_흐림               39609 non-null  bool 
 8   노면상태_기타               39609 non-null  bool 
 9   노면상태_서리/결빙            39609 non-null  bool 
 10  노면상태_적설               39609 non-null  bool 
 11  노면상태_젖음/습기            39609 non-null  bool 
 12  노면상태_침수               39609 non-null  bool 
 13  도로형태_교차로 - 교차로안       39609 non-null  bool 
 14  도로형태_교차로 - 교차로횡단보도내   39609 non-null  bool 
 15  도로형태_기타 - 기타          39609 non-nu

None

      Columns     


...

,day,hour,month,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_안개,기상상태_흐림,노면상태_기타,노면상태_서리/결빙,...,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,사고유형_차대차,사고유형_차량단독,사망자수,주말
5,1,4,1,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,0,0
24,2,10,1,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,0,0
29,2,13,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0,0
34,2,14,1,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,0,0
35,2,14,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39600,31,18,12,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,0,0
39601,31,18,12,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0,0
39603,31,19,12,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,0,0
39605,31,19,12,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,0,0


    Description   


,day,hour,month,사망자수,주말
count,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000
mean,15.817087,13.934863,6.678507,0.007776,0.247015
std,8.792314,5.626818,3.404229,0.090109,0.431281
min,1.000000,0.000000,1.000000,0.000000,0.000000
25%,8.000000,10.000000,4.000000,0.000000,0.000000
50%,16.000000,15.000000,7.000000,0.000000,0.000000
75%,23.000000,18.000000,10.000000,0.000000,0.000000
max,31.000000,23.000000,12.000000,2.000000,1.000000


      Unique      


day                     31
hour                    24
month                   12
기상상태_눈                   2
기상상태_맑음                  2
기상상태_비                   2
기상상태_안개                  2
기상상태_흐림                  2
노면상태_기타                  2
노면상태_서리/결빙               2
노면상태_적설                  2
노면상태_젖음/습기               2
노면상태_침수                  2
도로형태_교차로 - 교차로안          2
도로형태_교차로 - 교차로횡단보도내      2
도로형태_기타 - 기타             2
도로형태_단일로 - 고가도로위         2
도로형태_단일로 - 교량            2
도로형태_단일로 - 기타            2
도로형태_단일로 - 지하차도(도로)내     2
도로형태_단일로 - 터널            2
도로형태_미분류 - 미분류           2
도로형태_주차장 - 주차장           2
사고유형_차대차                 2
사고유형_차량단독                2
사망자수                     3
주말                       2
dtype: int64

In [19]:
wrangiling(use_test5)

       Shape      


(10963, 25)

        Info      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   day                   10963 non-null  int32
 1   hour                  10963 non-null  int32
 2   month                 10963 non-null  int32
 3   기상상태_눈                10963 non-null  bool 
 4   기상상태_맑음               10963 non-null  bool 
 5   기상상태_비                10963 non-null  bool 
 6   기상상태_흐림               10963 non-null  bool 
 7   노면상태_기타               10963 non-null  bool 
 8   노면상태_서리/결빙            10963 non-null  bool 
 9   노면상태_적설               10963 non-null  bool 
 10  노면상태_젖음/습기            10963 non-null  bool 
 11  노면상태_침수               10963 non-null  bool 
 12  도로형태_교차로 - 교차로안       10963 non-null  bool 
 13  도로형태_교차로 - 교차로횡단보도내   10963 non-null  bool 
 14  도로형태_기타 - 기타          10963 non-null  bool 
 15  도로형태_단일로 - 고가도로위      10963 non-nu

None

      Columns     


...

,day,hour,month,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_흐림,노면상태_기타,노면상태_서리/결빙,노면상태_적설,...,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,사고유형_차대차,사고유형_차량단독,주말
19,1,21,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1
41,2,20,1,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,1
53,3,12,1,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,0
56,3,13,1,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,0
58,3,13,1,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10909,29,22,12,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,0
10910,29,22,12,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,0
10913,29,23,12,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,0
10918,30,11,12,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0


    Description   


,day,hour,month,주말
count,10963.000000,10963.000000,10963.000000,10963.000000
mean,15.672900,13.980753,6.670984,0.255952
std,8.713575,5.501778,3.390321,0.436415
min,1.000000,0.000000,1.000000,0.000000
25%,8.000000,10.000000,4.000000,0.000000
50%,16.000000,15.000000,7.000000,0.000000
75%,23.000000,18.000000,10.000000,1.000000
max,31.000000,23.000000,12.000000,1.000000


      Unique      


day                     31
hour                    24
month                   12
기상상태_눈                   2
기상상태_맑음                  2
기상상태_비                   2
기상상태_흐림                  2
노면상태_기타                  2
노면상태_서리/결빙               2
노면상태_적설                  2
노면상태_젖음/습기               2
노면상태_침수                  2
도로형태_교차로 - 교차로안          2
도로형태_교차로 - 교차로횡단보도내      2
도로형태_기타 - 기타             2
도로형태_단일로 - 고가도로위         2
도로형태_단일로 - 교량            2
도로형태_단일로 - 기타            2
도로형태_단일로 - 지하차도(도로)내     2
도로형태_단일로 - 터널            2
도로형태_미분류 - 미분류           2
도로형태_주차장 - 주차장           2
사고유형_차대차                 2
사고유형_차량단독                2
주말                       2
dtype: int64

## 3. Modeling

In [20]:
## 3. hold-out train test split
x_train, x_valid, y_train, y_valid = train_test_split(use_train5[use_train5.columns.difference(['사망자수'])],
                                                      use_train5['사망자수'],
                                                      test_size = 0.3,
                                                      random_state = 42
                                                    )

In [21]:
# category values
labels = x_train.dtypes.reset_index()
categorical_cols = list(labels[labels[0] == 'category'].index)

In [22]:
print(x_train.dtypes)


day                     int32
hour                    int32
month                   int32
기상상태_눈                   bool
기상상태_맑음                  bool
기상상태_비                   bool
기상상태_안개                  bool
기상상태_흐림                  bool
노면상태_기타                  bool
노면상태_서리/결빙               bool
노면상태_적설                  bool
노면상태_젖음/습기               bool
노면상태_침수                  bool
도로형태_교차로 - 교차로안          bool
도로형태_교차로 - 교차로횡단보도내      bool
도로형태_기타 - 기타             bool
도로형태_단일로 - 고가도로위         bool
도로형태_단일로 - 교량            bool
도로형태_단일로 - 기타            bool
도로형태_단일로 - 지하차도(도로)내     bool
도로형태_단일로 - 터널            bool
도로형태_미분류 - 미분류           bool
도로형태_주차장 - 주차장           bool
사고유형_차대차                 bool
사고유형_차량단독                bool
주말                      int64
dtype: object


In [23]:
print(x_valid.dtypes)

day                     int32
hour                    int32
month                   int32
기상상태_눈                   bool
기상상태_맑음                  bool
기상상태_비                   bool
기상상태_안개                  bool
기상상태_흐림                  bool
노면상태_기타                  bool
노면상태_서리/결빙               bool
노면상태_적설                  bool
노면상태_젖음/습기               bool
노면상태_침수                  bool
도로형태_교차로 - 교차로안          bool
도로형태_교차로 - 교차로횡단보도내      bool
도로형태_기타 - 기타             bool
도로형태_단일로 - 고가도로위         bool
도로형태_단일로 - 교량            bool
도로형태_단일로 - 기타            bool
도로형태_단일로 - 지하차도(도로)내     bool
도로형태_단일로 - 터널            bool
도로형태_미분류 - 미분류           bool
도로형태_주차장 - 주차장           bool
사고유형_차대차                 bool
사고유형_차량단독                bool
주말                      int64
dtype: object


In [24]:
print(y_train.dtypes)

int64


In [25]:
print(y_valid.dtypes)

int64


In [26]:
# model
xgb = XGBRegressor(objective='count:poisson',
                    random_state = 42,
                    use_label_encoder=False,
                    enable_categorical=True,
                    tree_method='hist'
                    )
lgbm = LGBMRegressor(
    objective='poisson',
    random_state = 42
)
cb = CatBoostRegressor(
    cat_features = categorical_cols,
    objective = 'Poisson',
    random_state = 42
)

In [28]:
# fitting
xgb.fit(x_train, y_train)
lgbm.fit(x_train, y_train)
cb.fit(x_train, y_train,
        eval_set=(x_valid, y_valid),
        use_best_model = True,
        plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8647570	test: 0.8648429	best: 0.8648429 (0)	total: 193ms	remaining: 3m 12s
1:	learn: 0.7438881	test: 0.7439877	best: 0.7439877 (1)	total: 227ms	remaining: 1m 53s
2:	learn: 0.6468625	test: 0.6470725	best: 0.6470725 (2)	total: 289ms	remaining: 1m 35s
3:	learn: 0.5674146	test: 0.5676861	best: 0.5676861 (3)	total: 381ms	remaining: 1m 34s
4:	learn: 0.4999560	test: 0.5003715	best: 0.5003715 (4)	total: 425ms	remaining: 1m 24s
5:	learn: 0.4416079	test: 0.4421451	best: 0.4421451 (5)	total: 474ms	remaining: 1m 18s
6:	learn: 0.3915727	test: 0.3921964	best: 0.3921964 (6)	total: 570ms	remaining: 1m 20s
7:	learn: 0.3494641	test: 0.3502413	best: 0.3502413 (7)	total: 623ms	remaining: 1m 17s
8:	learn: 0.3126082	test: 0.3134890	best: 0.3134890 (8)	total: 659ms	remaining: 1m 12s
9:	learn: 0.2791780	test: 0.2801121	best: 0.2801121 (9)	total: 705ms	remaining: 1m 9s
10:	learn: 0.2518976	test: 0.2529316	best: 0.2529316 (10)	total: 799ms	remaining: 1m 11s
11:	learn: 0.2287289	test: 0.2298524	best:

In [29]:
# predict
pred_xgb = xgb.predict(x_valid)
pred_lgbm = lgbm.predict(x_valid)
pred_cb = cb.predict(x_valid)

In [30]:
# metrics
rmsle_xgb = mean_squared_log_error(y_valid, pred_xgb, squared = False)
rmsle_lgbm = mean_squared_log_error(y_valid, pred_lgbm, squared = False)
rmsle_cb = mean_squared_log_error(y_valid, pred_cb, squared = False)

print(f'xgboost : {rmsle_xgb}')
print(f'lightgbm : {rmsle_lgbm}')
print(f'catboost : {rmsle_cb}')

xgboost : 0.0637385810154068
lightgbm : 0.06390710618738714
catboost : 0.06301325674234952


/Users/shindonghyuk/opt/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/shindonghyuk/opt/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/shindonghyuk/opt/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/shindonghyuk/opt/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future vers